In [4]:

import nltk
from nltk.tree.tree import Tree
from nltk.corpus import semcor
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

nltk.download(['wordnet', 'semcor', 'omw-1.4', 'stopwords'])

import networkx as nx
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

[nltk_data] Downloading package wordnet to /home/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to /home/aditya/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aditya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from gensim.models import KeyedVectors
embeddings_index = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [6]:
x_train = []
y_train = []
count = 0
for sents in semcor.tagged_sents(tag="sem"):
  x = []
  y = []
  for w in sents:
    if isinstance(w, Tree) and isinstance(w[0], str):
      y.append(w.label())
      x.append(w[0])
  x_train.append(x)
  y_train.append(y)
  count += 1

In [7]:
corpus = []
for w in semcor.words():
  corpus.append(w)

In [8]:
def create_vectorizer(sentence_data_for_vocab, num_most_freq_words=20000, max_word_length=50):
  vectorizer = TextVectorization(max_tokens=num_most_freq_words, output_sequence_length=max_word_length, standardize='lower')
  text_ds = tf.data.Dataset.from_tensor_slices(sentence_data_for_vocab).batch(128)
  vectorizer.adapt(text_ds)
  voc = vectorizer.get_vocabulary()
  word_index = dict(zip(voc, range(len(voc))))
  return vectorizer, voc, word_index

In [9]:
vectorizer, voc, word_index = create_vectorizer(corpus)

2022-11-27 00:18:14.049939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 00:18:14.050200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-27 00:18:14.050245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-27 00:18:14.050280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-27 00:18:14.050315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [10]:
from keras.initializers import Constant
EMBEDDING_DIM = 300
MAX_NUM_WORDS = 20000
word_index = word_index
num_words = len(voc)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
count = 0
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    try:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix[i] = embedding_vector
        count += 1
    except (KeyError):
        continue
        
embedding_layer_google = tf.keras.layers.Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=50,
                            trainable=False)

In [11]:
p_cnt = 0
r_cnt = 0
num = 0
for idx in range(len(x_train)):
  if idx==1:
    break
  x = x_train[idx]
  y = y_train[idx]
  g = nx.Graph()
  defs = []
  vecs = []
  try:
    num += len(x)
    for word in x:
      t = np.array([[sns.definition()] for sns in wordnet.synsets(word)])
      vecs_loc = embedding_layer_google(vectorizer(t))
      vecs_loc = tf.keras.utils.normalize(tf.reduce_sum(vecs_loc, 1))
      defs.append([(sns.definition(), sns.lemmas()) for sns in wordnet.synsets(word)])
      vecs.append(vecs_loc)
    for w in defs:
      for d in w:
        g.add_node(d[0])
    for idx in range(len(defs)-1):
      w1 = defs[idx]
      w2 = defs[idx+1]
      for id1 in range(len(w1)):
        for id2 in range(len(w2)):
          weight = tf.tensordot(vecs[idx][id1], vecs[idx+1][id2], 1)
          g.add_edge(defs[idx][id1][0], defs[idx+1][id2][0], weight=tf.keras.backend.get_value(weight))
    # nx.draw(g)
    pr = nx.pagerank(g)
    senses = []
    for w in defs:
      max_pr = 0
      sense = None
      for d in w:
        if pr[d[0]] > max_pr:
          max_pr = pr[d[0]]
          sense = d[1]
      senses.append(sense)
    for idx in range(len(x)):
      if y[idx] in senses[idx]:
        p_cnt += 1
  except:
    pass
  

In [12]:
print("Accuracy =",p_cnt/num)

Accuracy = 0.7


In [13]:
text = 'he goes to a bank to deposit his money'

def sentTokenize(str):
    sentences = nltk.sent_tokenize(text)
    return sentences

In [14]:
senttoken = sentTokenize(text) 

In [15]:
def removeunnecessarywords(sentences2):
    stop_words = set(stopwords.words("english"))
    context_tab=[]
    for sentence in sentences2:
        words = nltk.word_tokenize(sentence)
        without_stop_words = [word for word in words if not word in stop_words]
        context_tab.append(without_stop_words)
    return context_tab 

In [16]:
remuness = removeunnecessarywords(senttoken)
testx = [item for sublist in  remuness for item in sublist]
print(testx)

['goes', 'bank', 'deposit', 'money']


In [17]:
g = nx.Graph()
defss = []
vecs = []
pt = 0
for word in testx:
  t = np.array([[sns.definition()] for sns in wordnet.synsets(word)])
  # print(word, t)
  try:
    vecs_loc = embedding_layer_google(vectorizer(t))
    vecs_loc = tf.keras.utils.normalize(tf.reduce_sum(vecs_loc, 1))
  except:
    print(" ")
  defss.append([(sns.definition(), sns.lemmas()) for sns in wordnet.synsets(word)])
  vecs.append(vecs_loc)
for w in defss:
  for d in w:
    g.add_node(d[0])
for idx in range(len(defss)-1):
  w1 = defss[idx]
  w2 = defss[idx+1]
  for id1 in range(len(w1)):
    for id2 in range(len(w2)):
      weight = tf.tensordot(vecs[idx][id1], vecs[idx+1][id2], 1)
      g.add_edge(defss[idx][id1][0], defss[idx+1][id2][0], weight=tf.keras.backend.get_value(weight))
# nx.draw(g)
pr = nx.pagerank(g)
senses = []
for w in defss:
  max_pr = 0
  sense = None
  for d in w:
    if pr[d[0]] > max_pr:
      max_pr = pr[d[0]]
      sense = d[1]
  senses.append(sense)
for i in senses:
  print(i)
assert len(testx) == len(senses)
# for idx in range(len(testx)):
#   print(testx[idx],":",senses[idx])
#   if testy[idx] in senses[idx]:
#     pt += 1
  
# print("Accuracy: ",pt/len(testx) )

[Lemma('belong.v.03.belong'), Lemma('belong.v.03.go')]
[Lemma('deposit.v.02.deposit'), Lemma('deposit.v.02.bank')]
[Lemma('deposit.v.02.deposit'), Lemma('deposit.v.02.bank')]
[Lemma('money.n.03.money')]
